# Closures

In [23]:
struct City {
    name: String,
    population: i64,
    country: String,
}

fn create_cities() -> Vec<City> {
    vec![
        City {
            name: "Tokyo".to_string(),
            population: 37_833_000,
            country: "Japan".to_string(),
        },
        City {
            name: "Delhi".to_string(),
            population: 29_399_141,
            country: "India".to_string(),
        },
        City {
            name: "Shanghai".to_string(),
            population: 26_317_104,
            country: "China".to_string(),
        },
        City {
            name: "São Paulo".to_string(),
            population: 21_650_000,
            country: "Brazil".to_string(),
        },
        City {
            name: "Cracow".to_string(),
            population: 769_498,
            country: "Poland".to_string(),
        },
    ]
}

let mut cities = create_cities();

cities.sort_by_key(|city| -city.population);

println!("Cities ordered by the population (descending):");
println!("-----------------------------------");
for city in &cities {
    println!("{}, {} - population: {}", city.name, city.country, city.population);
}


Cities ordered by the population (descending):
-----------------------------------
Tokyo, Japan - population: 37833000
Delhi, India - population: 29399141
Shanghai, China - population: 26317104
São Paulo, Brazil - population: 21650000
Cracow, Poland - population: 769498


()

## Closures that borrow

In [24]:
use std::collections::HashMap;

let mut cities_area = HashMap::new();

cities_area.insert("Tokyo".to_string(), 2187);
cities_area.insert("Delhi".to_string(), 1484);
cities_area.insert("Shanghai".to_string(), 6340);
cities_area.insert("São Paulo".to_string(), 1521);
cities_area.insert("Cracow".to_string(), 326);

In [25]:
cities.sort_by_key(|city| -cities_area[&city.name]);

In [26]:
println!("\nCities ordered by the area (descending):");
println!("-----------------------------------");
for city in &cities {
    println!("{}, {} - area: {} km²", city.name, city.country, cities_area[&city.name]);
}


Cities ordered by the area (descending):
-----------------------------------
Shanghai, China - area: 6340 km²
Tokyo, Japan - area: 2187 km²
São Paulo, Brazil - area: 1521 km²
Delhi, India - area: 1484 km²
Cracow, Poland - area: 326 km²


()

## Closures that move

In [27]:
use std::thread;

fn start_sorting_thread(mut cities: Vec<City>, cities_area: HashMap<String, i32>) -> thread::JoinHandle<Vec<City>> {
    let key_fn = move |city: &City| -> i32 { -cities_area[&city.name] };

    thread::spawn(move || {
        cities.sort_by_key(key_fn);
        cities
    })
}

start_sorting_thread(cities, cities_area).join().unwrap();

* Rust offers two ways for closures to get data from enclosing scopes: moves and borrowing
* If a closure would move a value of a copyable type, like `i32`, it copies the value instead
* Values of noncopyable types, like `Vec<City>`, really are moved: the preceding code transfers cities to the new thread, by way of the move closure 

## Function & Closure Types

### Function Type (functions only)

In [28]:
fn is_city_big(city: &City) -> bool {
    city.population > 10_000_000
}

* Function above has a type: `fn(&City) -> i64`

In [29]:
let city_predicate: fn(&City) -> bool = is_city_big;

In [30]:
let cities = create_cities();

city_predicate(&cities[0])

true

* We can pass function as an argument to another function.

In [31]:
fn count_selected_cities(cities: &Vec<City>, predicate: fn(&City) -> bool) -> i32 {
    let mut count = 0;
    for city in cities {
        if predicate(city) {
            count += 1;
        }
    }
    count

    // alternative one liner using iterator
    // cities.iter().filter(|city| predicate(city)).count() as i32
}

In [32]:
count_selected_cities(&cities, is_city_big)

4

### Fn Trait - functions & closures

In [33]:
let size_limit = 20_000_000;

let is_big_enough = |city: &City| -> bool { city.population > size_limit };

Error: closure may outlive the current function, but it borrows `size_limit`, which is owned by the current function

In [34]:
let size_limit = 20_000_000;

let is_big_enough: Box<dyn Fn(&City)->bool> = Box::new(|city: &City| -> bool { city.population > size_limit });

Error: `size_limit` does not live long enough

In [35]:
let size_limit = 20_000_000;

let is_big_enough: Box<dyn Fn(&City)->bool> = Box::new(move |city: &City| -> bool { city.population > size_limit });

In [36]:
count_selected_cities(&cities, is_big_enough)

Error: mismatched types

In [37]:
fn count_selected_cities(cities: &Vec<City>, predicate: impl Fn(&City) -> bool) -> i32 {
    let mut count = 0;
    for city in cities {
        if predicate(city) {
            count += 1;
        }
    }
    count
}

In [38]:
count_selected_cities(&cities, is_big_enough)

4

### FnOnce

In [45]:
let my_string = "Hello, World!".to_string();


Error: cannot move out of `my_string`, a captured variable in an `Fn` closure

In [47]:
fn call_twice<F>(closure: F) 
where F: Fn() {
    closure();
    closure();
}

In [48]:
let my_string = "Hello, World!".to_string();

call_twice(|| println!("{}", my_string));

Hello, World!
Hello, World!


In [49]:
call_twice(|| drop(my_string))

Error: cannot move out of `my_string`, a captured variable in an `Fn` closure

* Closures that drop values are not allowed to have `Fn`. They are, quite literally, no `Fn` at all. They implement a less powerful trait, `FnOnce`, the trait of closures that can be called once.

In [55]:
// mod explain {
//     trait Fn() -> R {
//         fn call(&self) -> R;
//     }

//     trait FnOnce() -> R {
//         fn call_once(self) -> R;
//     }
//     
//     trait FnMut() -> R {
//         fn call_mut(&mut self) -> R;
//     }
// }

* `Fn` is the family of closures and functions that you can call multiple times without restriction. This highest category also includes all `fn` functions.
* `FnMut` is the family of closures that can be called multiple times if the closure itself is declared mut.
* `FnOnce` is the family of closures that can be called once, if the caller owns the closure.

* `Fn()` is a subtrait of `FnMut()`, which is a subtrait of `FnOnce()`. 
* This makes `Fn` the most exclusive and most powerful category. `FnMut` and `FnOnce` are broader categories that include closures with usage restrictions.

## Copy and Clone for Closures

* Just as Rust automatically figures out which closures can be called only once, it can figure out which closures can implement `Copy` and `Clone`, and which cannot.

In [56]:
let y = 4;
let add_y = |x| x + y;
let copy_of_add_y = add_y; // closure is copied
assert_eq!(add_y(copy_of_add_y(3), 11);

Error: The variable `add_y` is a closure, which cannot be persisted.
You can however persist closures if you box them. e.g.:
let f: Box<dyn Fn()> = Box::new(|| {println!("foo")});
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.

# Functions & Closures

In [4]:
#[derive(Debug, PartialEq, Clone)]
struct Person {
    name: String,
    age: u8,
    height: f32
}

fn create_people() -> Vec<Person> {
    vec![
        Person {
            name: "Alice".to_string(),
            age: 20,
            height: 1.7
        },
        Person {
            name: "Bob".to_string(),
            age: 25,
            height: 1.8
        },
        Person {
            name: "Charlie".to_string(),
            age: 30,
            height: 1.9
        },
        Person {
            name: "David".to_string(),
            age: 35,
            height: 1.75
        },
        Person {
            name: "Eve".to_string(),
            age: 40,
            height: 1.75
        },
        Person {
            name: "Adam".to_string(),
            age: 16,
            height: 1.6
        },
        Person {
            name: "Marry".to_string(),
            age: 11,
            height: 1.5
        }
    ]
}

let people = create_people();

let people_sorted_by_age = {
    let mut people = people.clone();
    people.sort_by_key(|person| person.age);
    people
};

let people_sorted_by_height = {
    let mut people = people.clone();
    people.sort_by(|person, other|  f32::total_cmp(&person.height, &other.height));
    people
};

println!("People ordered by age: {:?}", people_sorted_by_age);
println!("People ordered by height: {:?}", people_sorted_by_height);

People ordered by age: [Person { name: "Marry", age: 11, height: 1.5 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]
People ordered by height: [Person { name: "Marry", age: 11, height: 1.5 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


## Closures Borrow by Default

In [7]:
use std::collections::HashMap;

fn create_name_translator() -> HashMap<String, String> {
    [
        ("Alice".to_string(), "Alicja".to_string()),
        ("Bob".to_string(), "Robert".to_string()),
        ("Charlie".to_string(), "Karol".to_string()),
        ("David".to_string(), "Dawid".to_string()),
        ("Eve".to_string(), "Ewa".to_string()),
        ("Adam".to_string(), "Adam".to_string()),
        ("Marry".to_string(), "Maria".to_string())
    ].iter().cloned().collect()
}

fn main() {
    let people = create_people();

    let name_translator: HashMap<_, _> = create_name_translator();

    let person_name_translator = |person: &Person| {
        let translated_name = name_translator.get(&person.name).unwrap_or(&person.name); // name_translator is borrowed
        Person {
            name: translated_name.to_string(),
            age: person.age,
            height: person.height
        }
    };

    let people_translated: Vec<Person> = people.iter().map(person_name_translator).collect();

    println!("People with translated names: {:?}", people_translated);
}

main();

People with translated names: [Person { name: "Alicja", age: 20, height: 1.7 }, Person { name: "Robert", age: 25, height: 1.8 }, Person { name: "Karol", age: 30, height: 1.9 }, Person { name: "Dawid", age: 35, height: 1.75 }, Person { name: "Ewa", age: 40, height: 1.75 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Maria", age: 11, height: 1.5 }]


## Closures That Move

In [8]:
fn main() {
    let people: Vec<Person> = create_people();
    let name_translator: HashMap<String, String> = create_name_translator();

    let person_name_translator = move |person: &Person| {
        let translated_name = name_translator.get(&person.name).unwrap_or(&person.name); // name_translator is moved
        Person {
            name: translated_name.to_string(),
            age: person.age,
            height: person.height
        }
    };

    let thd = std::thread::spawn(move || {    
        let people_translated: Vec<Person> = people.iter().map(person_name_translator).collect(); // people & person_name_translator are moved
    
        println!("People with translated names: {:?}", people_translated);
    });

    thd.join().unwrap();
}

main();

People with translated names: [Person { name: "Alicja", age: 20, height: 1.7 }, Person { name: "Robert", age: 25, height: 1.8 }, Person { name: "Karol", age: 30, height: 1.9 }, Person { name: "Dawid", age: 35, height: 1.75 }, Person { name: "Ewa", age: 40, height: 1.75 }, Person { name: "Adam", age: 16, height: 1.6 }, Person { name: "Maria", age: 11, height: 1.5 }]


## Function & Closure Types

### Function Type

In [9]:
fn is_adult(person: &Person) -> bool {
    person.age >= 18
}

The function `is_adult` has a type: `fn(&Person) -> bool`

In [10]:
let f_predicate: fn(&Person) -> bool = is_adult;

fn filter_by_predicate(people: &Vec<Person>, predicate: fn(&Person) -> bool) -> Vec<Person> {
    let mut qualified: Vec<Person> = Vec::new();
    for person in people {
        if predicate(person) {
            qualified.push(person.clone());
        }
    }
    qualified
}

let adults = filter_by_predicate(&people, is_adult);

println!("Adults: {:?}", adults);

Adults: [Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]


When function expects a function as an argument, it accepts a closure that captures nothing:

In [ ]:
let tall_people = filter_by_predicate(&people, |person| person.height > 1.75);

println!("Tall people: {:?}", tall_people);

Tall people: [Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


A closure that captures variables from the enclosing scope cannot be passed to a function that accepts a function type:

In [12]:
let height_limit = 1.75;
let tall_people = filter_by_predicate(&people, |person| person.height > height_limit);

Error: mismatched types

### `Fn` Trait

* Functions and closures implement the `Fn` trait, which is the trait of things that can be called like functions.

In [ ]:
fn filter_by_predicate<F>(people: &Vec<Person>, predicate: F) -> Vec<Person>
    where F: Fn(&Person) -> bool 
{
    let mut qualified: Vec<Person> = Vec::new();
    for person in people {
        if predicate(person) {
            qualified.push(person.clone());
        }
    }
    qualified
}

In [17]:
let adults = filter_by_predicate(&people, is_adult); // using function pointer
println!("Adults: {:?}", adults);

let height_limit = 1.75;
let tall_people = filter_by_predicate(&people, |person| person.height > height_limit); // using closure that captures (borrows) height_limit
println!("Tall people: {:?}", tall_people);

Adults: [Person { name: "Alice", age: 20, height: 1.7 }, Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }, Person { name: "David", age: 35, height: 1.75 }, Person { name: "Eve", age: 40, height: 1.75 }]
Tall people: [Person { name: "Bob", age: 25, height: 1.8 }, Person { name: "Charlie", age: 30, height: 1.9 }]


* Now the function `filter_by_predicate` is generic. It accepts any function or closure `F` that implements the special trait `Fn(&Person) -> bool`.
* The trait `Fn(&Person) -> bool` automatically implemented by all functions and closures that can be called with a `&Person` argument and return a `bool`.